This is the prototype of the Mean Field

There will be three main modules:

- 1) Agent

- 2) MeanField

- 3) BlackBoard

In [14]:
import numpy as np
import scipy as sp
import ode
from sliding_window import *

### Agent

In [24]:
class Agent:
    
    def __init__(self, state_indices, control_indices):
        '''
        state_indices (list of integers): This list tells which states pertain to this agent. e.g. [1,2] would 
        tell us that states 1 and 2 pertain to this agent.
        '''
        self.state_indices = state_indices
        self.control_indices = control_indices
        self.u_l_vec = None
        self.qp_l_vec = None
        self.u_l_dict = None
        self.qp_l_dict = None
    
    def L_l(self, q, q_dot, u):
        return
    
    def L_l_q_dot(self, t, q, q_dot, u, **kwargs):
        return
    
    def H_l_nou(self, q, p, lambda_l):
        return 

    def qp_rhs_l_nou(self, t, qp_vec, **kwargs):
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        u = kwargs['u_0']
        # for q-dot
        q_dot =  np.zeros(np.shape(p))
        # for p-dot
        p_dot =  np.zeros(np.shape(q))
        q_D_dot =  np.zeros(np.shape(p))
        p_D_dot =  np.zeros(np.shape(q))
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_l_nou(self, t, u_vec, **kwargs):
        qp_vec = kwargs['qp_vec']
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def H_l_u(self, q, p, u):
        return 
    
    def qp_rhs_l_u(self, t, qp_vec, **kwargs):
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        u = kwargs['u_0']
        # for q-dot
        q_dot =  np.zeros(np.shape(p))
        # for p-dot
        p_dot =  np.zeros(np.shape(q))
        q_D_dot =  np.zeros(np.shape(p))
        p_D_dot =  np.zeros(np.shape(q))
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_l_u(self, t, u_vec, **kwargs):
        qp_vec = kwargs['qp_vec']
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def qp_rhs_H_s(self, t, qp_vec, **kwargs):
        # This will include a term for qp_rhs_l and a term for qp_rhs_MF: 
        qp_vec
        qp_rhs_H_s = (1-alpha)*qp_rhs_l(self, qp_vec)+alpha*qp_rhs_MF()
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_H_s(self, t, u_vec, **kwargs):
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def H_l_D():
        pass
        
    def L_l_D():
        pass
        
    def L_l_D_q_Dot():
        pass
    # Inputs for numerical propagator
    q_0 = np.array([0])
    p_0 = np.array([0])
    q_D = np.array([0])
    p_D = np.array([0])
    u_0 = np.array([0])
    qpu_vec = np.hstack([q_0, p_0, q_D, p_D, u_0])
    state_dim = 1 
    Gamma = 1 
       
    # Inputs for numerical integration
    integrateTol = 10**-3
    integrateMaxIter = 40
       
    # Inputs for sliding window
    t_0 = 0 
    T =  2
    K=1 
    t_terminal = 2 
    n_s = 10



### MeanField

In [25]:
class MeanField:
    
    def __init__(self):
        pass
    
    def H_MF_nou(self, q, p):
        pass
        
    def qp_rhs_H_MF_nou(self, t, qp_vec, **kwargs):
        pass

    def H_MF_u(self, q, p):
        pass
        
    def qp_rhs_H_MF_u(self, t, qp_vec, **kwargs):
        pass
          

### Blackboard

In [35]:
class Blackboard:
    
    def __init__(self):
        '''
#         state_agent_mapping is dictionary with keys: object of class Agent, 
#               values: list of integers state_indices state_indices
        '''
#         self.state_agent_mapping={}
        '''       
        q_p_u_dict is a dictionary which maps 'q', 'p', 'u', to a dictionary of index-value pairs: local values for q, p, and u for this agent.  
                        blackboard holds all of the most recent local values, e.g.
                        {'q': {'1':3, '2':0}, 'p': {'1':3, '2': 2}, 'u': {'1': 0}}
                        It doesn't care which agent updated them most recently.  It only needs to know which values to update.
        q_p_u_dict initially will be filled 
        '''
        self.q_p_u_dict={'q':{}, 'p':{}, 'u':{}}
    
    def construct_p_q(self, agent):
        '''
        Inputs:
            agent (instance of class Agent): This is the agent for which we are constructing p and q.
        
        Outputs:
            qp_vec (np.array): vector of q and p values
            u_vec (np.array): vector of u values
        '''
        # find which states pertain to this agent
        # for the states that don't, get the values from q_p_u_dict and put them into qp_vec 
        
        qp_vec = agent
        # read all other values for other states from agent_q_p_u_dict
        return qp_vec, u_vec
    
    def update_q_p_u_dict(self, agent):
        '''  This method should be called after local propagation of each agent
        Inputs:
            agent (instance of class Agent): this is the agent whos values we are updating
        Outputs:
            No outputs.  This method just updates the attributes of the blackboard,
            just update the dictionary, agent_q_p_u_dict.
        '''
        # determine which states pertain to this agent and replace the old values with new
        for state_ix in agent.state_indices:
            self.q_p_u_dict['q'][str(state_ix)] = agent.q_p_u_dict['q'][str(state_ix)]
            self.q_p_u_dict['p'][str(state_ix)] = agent.q_p_u_dict['p'][str(state_ix)]
            
        for control_ix in agent.control_indices:
            self.q_p_u_dict['u'][str(control_ix)] = agent.q_p_u_dict['u'][str(control_ix)]
        
    # Look at what things the Blackboard needs
    # It at least needs a mapping that tells which states pertain to which agent
    # It needs to keep track of all states, costates, controls for all agents so that we can plug in values when necessary

#### small test

In [38]:
myAgent=Agent(state_indices=[1,2], control_indices=[1,3])
myAgent.q_p_u_dict={}
myAgent.q_p_u_dict['q']={}
myAgent.q_p_u_dict['q']['1']=[0]
myAgent.q_p_u_dict['q']['2']=[0]

myAgent.q_p_u_dict['p']={}
myAgent.q_p_u_dict['p']['1']=[0]
myAgent.q_p_u_dict['p']['2']=[0]


myAgent.q_p_u_dict['u']={}
myAgent.q_p_u_dict['u']['1']=[0]
myAgent.q_p_u_dict['u']['2']=[0]

bb=Blackboard()


In [39]:
bb.update_q_p_u_dict(myAgent)

KeyError: '3'

#### What things we need in the blackboard: